# Import libraries and data

In [1]:
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [4]:
# Get annotated tweets
tweets = pd.read_csv('../data/local/labelled_tweets_clean.csv').reset_index()
#tweets['prompt'] = "Please give me the sentiment regarding vaccination in the following tweet: \"TWEET_TEXT_HERE\"\nUse one of the following words: neutral, negative, positive"
#tweets['prompt'] = tweets.apply(lambda x: x['prompt'].replace('TWEET_TEXT_HERE', str(x['text'])), axis = 1)
#tweets.head()
tweets_new = pd.read_csv('../data/local/labelled_tweets_new_clean.csv').reset_index()
tweets_new.head()

,index,id,text,label_tag
0,0,1008450587237797888,just saw a guy wear a shirt that says “thot va...,neutral
1,1,1008751928568840192,"Great news to see. Hopefully, folks who disput...",positive
2,2,1008751948957351936,The global community must invest in further re...,positive
3,3,1008752099222523904,HPV vaccine has almost wiped out infections in...,positive
4,4,1010156916553736192,So I can catch this shit too and need treatmen...,negative


# Using three categories for vaccine sentiment

In [4]:
# Prompt for GPT
prompt_raw = "Please give me the sentiment regarding vaccination in the following tweet: \"TWEET_TEXT_HERE\"\n
Use one of the following words: neutral, negative, positive"

In [5]:
#prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets.iloc[0,1])

In [5]:
len(tweets.index)

5321

In [34]:
len(tweets_new.index)

185

In [7]:
test = tweets.iloc[2966:, :]
test.info

<bound method DataFrame.info of       index                   id  \
2966   2966  1337444266210168832   
2967   2967  1337444446749732864   
2968   2968  1337445253901643776   
2969   2969  1337452595095490560   
2970   2970  1337458140577583104   
...     ...                  ...   
5316   5316  1355647818560659456   
5317   5317  1355657080380788736   
5318   5318  1355657313168678912   
5319   5319  1355657442479235072   
5320   5320  1355664293534998528   

                                                   text label_tag  
2966  The history of #racism in healthcare is real a...   neutral  
2967  Coronavirus vaccine line depends largely on th...   neutral  
2968  just been offered the covid vaccine at work an...   neutral  
2969  Oddly enough, large shipments of vaccines that...   neutral  
2970  POLL: Will you take the COVID vaccine when it ...   neutral  
...                                                 ...       ...  
5316  We have thousands of 1A and 1B and 1C that hav...   n

In [7]:
tweets1 = tweets.iloc[0:306,:]
tweets2 = tweets.iloc[306:601,:]
tweets3 = tweets.iloc[601:901, :]
tweets4 = tweets.iloc[901:1201, :]


In [103]:
tweets13 = tweets.iloc[2670:,:]

In [31]:
id_gpt = []
sent_gpt = []

In [35]:
for i in range(0,185):
    prompt_i = prompt_raw.replace('TWEET_TEXT_HERE', tweets_new.iloc[i,2])
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_i,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt.append(tweets_new.iloc[i,0])
    sent_gpt.append(response.choices[0].text)

In [36]:
df_gpt_total = pd.DataFrame(list(zip(id_gpt, sent_gpt)),
                            columns = ['id_gpt', 'sentiment_gpt'])
df_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_gpt         185 non-null    int64 
 1   sentiment_gpt  185 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.0+ KB


In [37]:
df_gpt_total.tail()

,id_gpt,sentiment_gpt
180,180,\n\nPositive
181,181,\nNegative
182,182,\n\nPositive
183,183,\n\nneutral
184,184,\n\nNegative


In [108]:
id_gpt13 = id_gpt
sent_gpt13 = sent_gpt
df_gpt_total13 = df_gpt_total

In [25]:
#df_gpt_total1.to_csv('../data/local/gpt_sentiment1.csv')
df_gpt_total1['sentiment_gpt2'] = df_gpt_total1['sentiment_gpt'].to_string()
print(type(df_gpt_total1['sentiment_gpt2']))

<class 'pandas.core.series.Series'>


In [30]:
#sent_gpt1.to_csv('../data/local/gpt_sentiment1_only.csv')

In [38]:
df_gpt_total.to_excel('../data/local/gpt_sentiment_newest.xlsx')


# Using two categories for vaccine sentiment

In [6]:
tweets_pos_neg = tweets[tweets['label_tag'].isin(['positive', 'negative'])]
tweets_pos_neg.head()

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive
26,26,1237492335430250496,@user @user thank you for the vaccine LOL #Wel...,positive
28,28,1237894709306986496,Remember polio? Killed a lot of people. Vaccin...,positive
38,38,1239077015250030592,You be ready to pay what this Coronavirus vacc...,negative


In [39]:
tweets_new_pos_neg = tweets_new[tweets_new['label_tag'].isin(['positive', 'negative'])]
tweets_new_pos_neg.head()

,index,id,text,label_tag
1,1,1008751928568840192,"Great news to see. Hopefully, folks who disput...",positive
2,2,1008751948957351936,The global community must invest in further re...,positive
3,3,1008752099222523904,HPV vaccine has almost wiped out infections in...,positive
4,4,1010156916553736192,So I can catch this shit too and need treatmen...,negative
5,5,1010157450618601472,Could a #vaccine for #type-1 diabetes be on th...,positive


In [42]:
prompt_raw_pos_neg = "Please give me the sentiment regarding vaccination in the following tweet: \
"TWEET_TEXT_HERE\"\nUse one of the following words: negative, positive"

In [40]:
len(tweets_new_pos_neg.index)

128

In [9]:
test = tweets_pos_neg.iloc[0:2, :]
test

,index,id,text,label_tag
18,18,1235574994924490752,@user @user I saw #VaxxedII yesterday and I ha...,negative
22,22,1236451808412909568,Dear anti-vaxxers. If you are legitimately wor...,positive


In [7]:
tweets_pos_neg1 = tweets_pos_neg.iloc[0:306,:]
tweets_pos_neg2 = tweets_pos_neg.iloc[306:601,:]
tweets_pos_neg3 = tweets_pos_neg.iloc[601:901, :]
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]


In [103]:
tweets_pos_neg4 = tweets_pos_neg.iloc[901:1201, :]

In [43]:
id_gpt_two = []
sent_gpt_two = []

In [44]:
for i in range(0,128):
    prompt_two_i = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_new_pos_neg.iloc[i,2])
    response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_i,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    id_gpt_two.append(tweets_new_pos_neg.iloc[i,0])
    sent_gpt_two.append(response_two.choices[0].text)

In [17]:
prompt_two_1339 = prompt_raw_pos_neg.replace('TWEET_TEXT_HERE', tweets_pos_neg.iloc[1339,2])
response_two = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt_two_1339,
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
id_gpt_two.append(tweets_pos_neg.iloc[1339,0])
sent_gpt_two.append(response_two.choices[0].text)

In [46]:
df_two_gpt_total = pd.DataFrame(list(zip(id_gpt_two, sent_gpt_two)),
                            columns = ['id_gpt_two', 'sentiment_gpt_two'])
df_two_gpt_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_gpt_two         128 non-null    int64 
 1   sentiment_gpt_two  128 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


In [47]:
df_two_gpt_total.tail()

,id_gpt_two,sentiment_gpt_two
123,179,", or neutral\n\nPositive"
124,180,", or neutral\n\nPositive"
125,181,", or neutral\n\nNegative"
126,182,", or neutral\n\nPositive"
127,184,\n\nNegative


In [108]:
id_gpt13 = id_gpt
sent_gpt13 = sent_gpt
df_gpt_total13 = df_gpt_total

In [25]:
#df_gpt_total1.to_csv('../data/local/gpt_sentiment1.csv')
df_gpt_total1['sentiment_gpt2'] = df_gpt_total1['sentiment_gpt'].to_string()
print(type(df_gpt_total1['sentiment_gpt2']))

<class 'pandas.core.series.Series'>


In [30]:
#sent_gpt1.to_csv('../data/local/gpt_sentiment1_only.csv')

In [48]:
df_two_gpt_total.to_excel('../data/local/gpt_sentiment_new_pos_neg.xlsx')
